In [9]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from pprint import pprint

## Loading Data


In [10]:
df = pd.read_csv('train.csv') 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [12]:
X = df.drop(['count', 'casual', 'registered'], axis=1) 
y = df['count']
#drop 'casual', 'registered here because test.csv doesn't have these columns

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [14]:
def extract(df):
        df.datetime = pd.to_datetime(df.datetime)
        df['day'] = df.datetime.dt.day
        df['hour'] = df.datetime.dt.hour
        df['weekday'] = df.datetime.dt.weekday
        df['year'] = df.datetime.dt.year
        df['month'] = df.datetime.dt.month
        
        return pd.concat([df[['day']],pd.get_dummies(df[['year','month','weekday','hour']],columns=['year','month','weekday','hour'])],axis=1)



## Create pipeline

In [15]:
preprosessor = ColumnTransformer([
    ('do_nothing', 'passthrough', ['holiday', 'workingday', 'season', 'weather', 'windspeed', 'temp', 'humidity']),
    ('time_extact', FunctionTransformer(extract), ['datetime']),
    #('one_hot_encoding', OneHotEncoder(sparse = False), ['season','weather']),
    #('0_imputer', SimpleImputer(strategy='mean', fill_value=0), ['windspeed']),
    ], 
    remainder='drop')  

In [16]:
# create the model pipeline
pipeline = make_pipeline(preprosessor, RandomForestRegressor())

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                                  ['holiday', 'workingday',
                                                   'season', 'weather',
                                                   'windspeed', 'temp',
                                                   'humidity']),
                                                 ('time_extact',
                                                  FunctionTransformer(func=<function extract at 0x7fd4323e2040>),
                                                  ['datetime'])])),
                ('randomforestregressor', RandomForestRegressor())])

In [18]:
X_train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
2930,2011-07-11 00:00:00,3,0,1,1,28.70,32.575,65,12.9980
7669,2012-05-18 22:00:00,2,0,1,1,22.96,26.515,52,22.0028
1346,2011-04-01 23:00:00,2,0,1,1,12.30,15.910,61,6.0032
9432,2012-09-16 09:00:00,3,0,0,1,23.78,27.275,60,8.9981
453,2011-02-01 23:00:00,1,0,1,3,8.20,9.850,93,12.9980
...,...,...,...,...,...,...,...,...,...
5734,2012-01-14 02:00:00,1,0,0,1,6.56,8.335,47,11.0014
5191,2011-12-10 09:00:00,4,0,0,1,11.48,12.880,61,19.0012
5390,2011-12-18 16:00:00,4,0,0,1,11.48,13.635,48,16.9979
860,2011-02-19 07:00:00,1,0,0,1,15.58,19.695,17,35.0008


In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8164 entries, 2930 to 7270
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    8164 non-null   object 
 1   season      8164 non-null   int64  
 2   holiday     8164 non-null   int64  
 3   workingday  8164 non-null   int64  
 4   weather     8164 non-null   int64  
 5   temp        8164 non-null   float64
 6   atemp       8164 non-null   float64
 7   humidity    8164 non-null   int64  
 8   windspeed   8164 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 637.8+ KB


In [20]:
pipeline.score(X_train, y_train)

0.9884144310094124

In [21]:
pipeline.score(X_test, y_test)

0.9270697104992377

## Cross Validation

In [22]:
pipeline = make_pipeline(preprosessor, RandomForestRegressor()) #an unfitted model, default value

In [23]:
cross_acc = cross_val_score(estimator=pipeline,        # estimator: # the model you want to evaluate/unfitted 
                            X=X_train,            # the training input data 
                            y=y_train,            # the training output data  
                            cv=5,                # number of cross validation datasets 
                            scoring='r2'   # evaluation metric 
                            ) 

In [24]:
cross_acc

array([0.90130429, 0.91003028, 0.91205443, 0.91732675, 0.90194473])

In [25]:
cross_acc.mean()   

0.9085320975666425

## Grid Search Tuning

In [26]:
pipeline_opt = make_pipeline(preprosessor, RandomForestRegressor())  #an unfitted model

In [27]:
pprint(pipeline.get_params())

{'columntransformer': ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                 ['holiday', 'workingday', 'season', 'weather',
                                  'windspeed', 'temp', 'humidity']),
                                ('time_extact',
                                 FunctionTransformer(func=<function extract at 0x7fd4323e2040>),
                                 ['datetime'])]),
 'columntransformer__do_nothing': 'passthrough',
 'columntransformer__n_jobs': None,
 'columntransformer__remainder': 'drop',
 'columntransformer__sparse_threshold': 0.3,
 'columntransformer__time_extact': FunctionTransformer(func=<function extract at 0x7fd4323e2040>),
 'columntransformer__time_extact__accept_sparse': False,
 'columntransformer__time_extact__check_inverse': True,
 'columntransformer__time_extact__func': <function extract at 0x7fd4323e2040>,
 'columntransformer__time_extact__inv_kw_args': None,
 'columntransformer__time_extact__inverse_func': None,
 

In [28]:
# define our hyperparameters to combine
hyperparam_grid = {
    'randomforestregressor__max_depth': [2, 5, 10, 20, 30],
    'randomforestregressor__n_estimators': [5, 10, 50, 100, 200],
    'randomforestregressor__min_samples_leaf': [1,5,10]
}


In [29]:
grid_cv = GridSearchCV(estimator=pipeline_opt,            # unfitted model/estimator
                       param_grid=hyperparam_grid,  # hyperparameters dict
                       cv=5,                        # number of folds, k
                       scoring='r2')                # scoring metric

In [30]:
# fit all models with all the different hyperparamters
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('do_nothing',
                                                                         'passthrough',
                                                                         ['holiday',
                                                                          'workingday',
                                                                          'season',
                                                                          'weather',
                                                                          'windspeed',
                                                                          'temp',
                                                                          'humidity']),
                                                                        ('time_extact',
                                                          

In [31]:
results_df = pd.DataFrame(grid_cv.cv_results_)

In [32]:
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035107,0.003145,0.011078,0.002149,2,1,5,"{'randomforestregressor__max_depth': 2, 'rando...",0.278069,0.300570,0.235372,0.304196,0.249502,0.273542,0.027297,74
1,0.049195,0.004407,0.009001,0.001542,2,1,10,"{'randomforestregressor__max_depth': 2, 'rando...",0.261121,0.294137,0.283530,0.269602,0.271935,0.276065,0.011528,73
2,0.162326,0.001711,0.023491,0.029615,2,1,50,"{'randomforestregressor__max_depth': 2, 'rando...",0.294006,0.296984,0.269868,0.299229,0.276618,0.287341,0.011824,61
3,0.310498,0.001868,0.011299,0.000229,2,1,100,"{'randomforestregressor__max_depth': 2, 'rando...",0.292225,0.294134,0.264817,0.295032,0.279070,0.285056,0.011652,65
4,0.611883,0.002628,0.016164,0.000385,2,1,200,"{'randomforestregressor__max_depth': 2, 'rando...",0.294455,0.300216,0.266549,0.298676,0.276356,0.287250,0.013400,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.095163,0.002687,0.007244,0.000234,30,10,5,"{'randomforestregressor__max_depth': 30, 'rand...",0.783335,0.833604,0.812973,0.829210,0.808002,0.813425,0.017840,29
71,0.180974,0.005405,0.008099,0.000242,30,10,10,"{'randomforestregressor__max_depth': 30, 'rand...",0.803229,0.850079,0.827084,0.837474,0.830455,0.829664,0.015387,26
72,0.843722,0.007807,0.015517,0.000569,30,10,50,"{'randomforestregressor__max_depth': 30, 'rand...",0.811048,0.845921,0.835874,0.846339,0.825438,0.832924,0.013358,23
73,1.674000,0.015711,0.023936,0.000135,30,10,100,"{'randomforestregressor__max_depth': 30, 'rand...",0.812636,0.844775,0.842503,0.846415,0.827817,0.834829,0.012913,21


In [33]:
results_df.shape

(75, 16)

In [34]:
results_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_randomforestregressor__max_depth',
       'param_randomforestregressor__min_samples_leaf',
       'param_randomforestregressor__n_estimators', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')

In [35]:
results_df[['param_randomforestregressor__max_depth', 'param_randomforestregressor__min_samples_leaf', 'param_randomforestregressor__n_estimators', 'mean_test_score',
       'std_test_score', 'rank_test_score']].sort_values('rank_test_score')

,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__n_estimators,mean_test_score,std_test_score,rank_test_score
64,30,1,200,0.909588,0.005720,1
63,30,1,100,0.909077,0.005951,2
62,30,1,50,0.908450,0.005434,3
49,20,1,200,0.897349,0.006025,4
47,20,1,50,0.896268,0.004916,5
...,...,...,...,...,...,...
10,2,10,5,0.280664,0.014483,71
6,2,5,10,0.278929,0.011852,72
1,2,1,10,0.276065,0.011528,73
0,2,1,5,0.273542,0.027297,74


In [36]:
grid_cv.best_params_  

{'randomforestregressor__max_depth': 30,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__n_estimators': 200}

In [37]:
#estimating the test performance after tuning
pipeline_best = make_pipeline(preprosessor,RandomForestRegressor(max_depth=30, n_estimators=200, min_samples_leaf=1))

cross_acc = cross_val_score(estimator=pipeline_best,  # estimator: # the model you want to evaluate 
                            X=X_train,          # the training input data 
                            y=y_train,          # the training output data  
                            cv=5,               # number of cross validation datasets, k   
                            scoring='r2')       # evaluation metric 

In [38]:
cross_acc

array([0.90298346, 0.91201935, 0.9125163 , 0.91819552, 0.90572925])

In [39]:
cross_acc.mean()

0.9102887754081846

In [40]:
pipeline_best.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                                  ['holiday', 'workingday',
                                                   'season', 'weather',
                                                   'windspeed', 'temp',
                                                   'humidity']),
                                                 ('time_extact',
                                                  FunctionTransformer(func=<function extract at 0x7fd4323e2040>),
                                                  ['datetime'])])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=30, n_estimators=200))])

In [41]:
pipeline_best.score(X_train, y_train)

0.9883626427319889

In [42]:
pipeline_best.score(X_test, y_test)

0.9260027735783412

## RMSLE

In [43]:
def rmsle(y_pred, y,convertExp=True):
    if convertExp:
        y_pred = np.exp(y_pred),
        y = np.exp(y)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y_pred]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [44]:
y_pred = pipeline_best.predict(X_train) 

In [45]:
rmsle(y_pred, y_train, convertExp=True)

#doesn't work

/var/folders/fs/bmr1lyws1dx3sp07l0gytvmc0000gn/T/ipykernel_43730/1284209182.py:3: RuntimeWarning: overflow encountered in exp
  y_pred = np.exp(y_pred),
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/fs/bmr1lyws1dx3sp07l0gytvmc0000gn/T/ipykernel_43730/1284209182.py:7: RuntimeWarning: overflow encountered in square
  calc = (log1 - log2) ** 2


inf

## Submission...

In [46]:
kaggle_data = pd.read_csv('test.csv')

In [47]:
kaggle_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014
...,...,...,...,...,...,...,...,...,...
6488,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014
6489,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014
6490,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014
6491,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981


In [48]:
predictions = pipeline_best.predict(kaggle_data)

In [49]:
submission = pd.DataFrame({'datetime':kaggle_data['datetime'],'count':predictions})

In [50]:
submission

,datetime,count
0,2011-01-20 00:00:00,20.588089
1,2011-01-20 01:00:00,15.294776
2,2011-01-20 02:00:00,14.398538
3,2011-01-20 03:00:00,8.667545
4,2011-01-20 04:00:00,7.254925
...,...,...
6488,2012-12-31 19:00:00,217.731402
6489,2012-12-31 20:00:00,154.803691
6490,2012-12-31 21:00:00,122.558156
6491,2012-12-31 22:00:00,111.428315


In [51]:
#Convert DataFrame to a csv file that can be uploaded
filename = 'Bike Sharing Demand RF.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)        

Saved file: Bike Sharing Demand RF.csv
